In [11]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn as nn

# Load tokenizer and pretrained DepRoBERTa model
model_name = "rafalposwiata/deproberta-large-v1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
backbone = AutoModel.from_pretrained(model_name)

# Freeze the base model — we’re not retraining it
for param in backbone.parameters():
    param.requires_grad = False

# Simple predictive head — 1 neuron for regression
class DepressionPredictor(nn.Module):
    def __init__(self, backbone):
        super().__init__()
        self.backbone = backbone
        hidden_size = backbone.config.hidden_size
        self.head = nn.Sequential(
            nn.Linear(hidden_size, 256),
            nn.ReLU(),
            nn.Linear(256, 1),
            nn.Sigmoid()  # ensures output between 0 and 1
        )

    def forward(self, input_ids, attention_mask):
        outputs = self.backbone(input_ids=input_ids, attention_mask=attention_mask)
        pooled = outputs.last_hidden_state[:, 0]  # [CLS] token representation
        return self.head(pooled)

# Instantiate model
model = DepressionPredictor(backbone)
model.eval()

# Example prediction
text = "I can’t focus on anything lately."
inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
with torch.no_grad():
    score = model(**inputs).item()

print(f"text: {text}")
print(f"predicted depression score: {score:.4f}")


Some weights of RobertaModel were not initialized from the model checkpoint at rafalposwiata/deproberta-large-v1 and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


text: I can’t focus on anything lately.
predicted depression score: 0.3902
